### En este notebook procedemos a realizar la limpieza de los DataFrames extraídos

**Índice**

*DataFrame Valores de Mercado*
1. [Unimos DataFrames de Valores de Mercado](#id1)
2. [Colocamos el año de inicio de la temporada en Valores de Mercado](#id2)

*DataFrame Estadísticas*
1. [Añadimos columna de Internacionalidades a los Porteros](#id3)
2. [Limpiamos la columna "Temporada"](#id4)
3. [Limpiamos la columna "Nombre"](#id5)
4. [Limpiamos la columna "Internacionalidades/Goles"](#id6)
5. [Limpiamos la columna "Minutos Jugados"](#id7)
6. [Últimos retoques a los DataFrames](#id8)
7. [Vemos los DataFrames limpios](#id9)
8. [Unimos los DataFrames de Porteros y Jugadores(resto)](#id10)

*Retoques finales y exportación a csv*

1. [Añadir posición y nacionalidad de los jugadores al DataFrame](#id11)
2. [Exportación final de lo relevante](#id12)


In [2]:
import pandas as pd
import numpy as np
import openpyxl
import re

<div id='id1' />

##### Unimos los DataFrames de subidas de valor y de bajadas de valor

In [44]:
df_market_down = pd.read_excel('../WebScraping/Selenium/data/Selenium_Bajadas.xlsx',index_col=0).reset_index(drop=True)
df_market_up = pd.read_excel('../WebScraping/Selenium/data/Selenium_Subidas.xlsx',index_col=0).reset_index(drop=True)

In [45]:
df_market  = pd.concat([df_market_down,df_market_up],axis=0)

<div id='id2' />

##### Un dato importante a destacar es saber en qué temporada estamos

Vamos a establecer que, a partir de julio de cada año, cambia la temporada en la que se juega. No sucede en todas las ligas, pero tiene una autenticidad mayor que decir que el año en el que se juega es el año de la temporada

Sacamos los meses

In [46]:
lista_mes = []
for i in df_market['Fecha de actualización']:
    lista_mes.append(int(str(i)[5:7]))

df_market['Mes'] = lista_mes

Creamos la columna temporada

In [47]:
lista_resta = []
for i in df_market['Mes']:
    if i <7:
        lista_resta.append(-1)
    else:
        lista_resta.append(0)

df_market['Resta'] = lista_resta
df_market['Temporada'] = df_market['Año de actualización']+df_market['Resta']

In [48]:
df_market

,Nombre,Fecha de actualización,Valor de mercado,Club,Edad,Diferencia_Valor,Club_Valor,Año de actualización,Mes,Resta,Temporada
0,Romelu Lukaku,2023-03-21,40000000,Inter de Milán,29,-15000000,Inter de Milán,2023,3,-1,2022
1,Romelu Lukaku,2022-11-11,55000000,Inter de Milán,29,-15000000,Chelsea FC,2022,11,0,2022
2,Romelu Lukaku,2022-06-15,70000000,Chelsea FC,29,-15000000,Chelsea FC,2022,6,-1,2021
3,Romelu Lukaku,2022-03-29,85000000,Chelsea FC,28,-15000000,Chelsea FC,2022,3,-1,2021
4,Romelu Lukaku,2021-12-23,100000000,Chelsea FC,28,0,Inter de Milán,2021,12,0,2021
...,...,...,...,...,...,...,...,...,...,...,...
54020,Vladan Danilovic,2019-03-19,225000,FK Borac Banja Luka,19,-25000,FK Borac Banja Luka,2019,3,-1,2018
54021,Vladan Danilovic,2018-09-06,250000,FK Borac Banja Luka,19,-100000,FK Borac Banja Luka,2018,9,0,2018
54022,Vladan Danilovic,2018-02-15,350000,FK Borac Banja Luka,18,100000,FK Borac Banja Luka,2018,2,-1,2017
54023,Vladan Danilovic,2017-07-20,250000,FK Borac Banja Luka,17,100000,FK Borac Banja Luka U19,2017,7,0,2017


***

# Limpiamos los DataFrames de las Estadísticas

<div id='id3' />

##### Añadimos columna de Internacionalidades a los Porteros

In [49]:
df_stats_players = pd.read_excel('../WebScraping/Octoparse/data/Rendimiento_Jugadores.xlsx',index_col=0).reset_index(drop=True)
df_stats_gk = pd.read_excel('../WebScraping/Octoparse/data/Rendimiento_Porteros.xlsx',index_col=0).reset_index(drop=True)

In [50]:
internacionalidades_df_gk = pd.read_excel('../WebScraping/Octoparse/data/Internacionalidades_porteros.xlsx')
internacionalidades_df_gk

,Nombre,Internacionalidades
0,#16 \n ...,Partidos internacionales/goles:\n ...
1,#1 \n ...,Partidos internacionales/goles:\n ...
2,#13 \n ...,Partidos internacionales/goles:\n ...
3,#21 \n ...,Partidos internacionales/goles:\n ...
4,#1 \n ...,Partidos internacionales/goles:\n ...
...,...,...
530,#13 \n ...,Partidos internacionales/goles:\n ...
531,#1 \n ...,Partidos internacionales/goles:\n ...
532,#1 \n ...,Partidos internacionales/goles:\n ...
533,#30 \n ...,Partidos internacionales/goles:\n ...


In [51]:
df_stats_gk

,Temporada,Liga,Club,Partidos_Jugados,Goles_en_contra,Porterias_a_cero,Minutos_Jugados,Nombre,Altura,Competicion
0,22/23,Premier League,Chelsea FC,9,12,1,764',#16 \n ...,"1,94 m",LIGAS NACIONALES
1,21/22,Premier League,Chelsea FC,34,31,14,3.060',#16 \n ...,"1,94 m",LIGAS NACIONALES
2,20/21,Premier League,Chelsea FC,31,25,16,2.746',#16 \n ...,"1,94 m",LIGAS NACIONALES
3,20/21,Ligue 1,Stade Rennais FC,1,1,-,90',#16 \n ...,"1,94 m",LIGAS NACIONALES
4,19/20,Ligue 1,Stade Rennais FC,24,19,9,2.160',#16 \n ...,"1,94 m",LIGAS NACIONALES
...,...,...,...,...,...,...,...,...,...,...
7954,15/16,LaLiga SmartBank,SD Huesca,15,22,1,1.304',#13 \n ...,NaN,NaN
7955,14/15,Promoción a LaLiga,Real Zaragoza,1,3,-,90',#13 \n ...,NaN,NaN
7956,14/15,LaLiga SmartBank,Real Zaragoza,19,28,4,1.710',#13 \n ...,NaN,NaN
7957,13/14,LaLiga SmartBank,Real Zaragoza,2,2,-,180',#13 \n ...,NaN,NaN


In [52]:
df_stats_gk = df_stats_gk.merge(internacionalidades_df_gk, how='inner', on = 'Nombre')

<div id='id4' >

## Temporada

Tendremos en cuenta el año de comienzo de temporada, al igual que en el DataFrame de Valores de Mercado


#### Jugadores (sin contar porteros)

In [53]:
df_stats_players['Temporada'] = [re.sub(r"\d\d/\d\d", '20'+str(i)[-2:], str(i)) for i in df_stats_players['Temporada']]

temporadas_year_int = []
for i in df_stats_players['Temporada']:
    try:
        temporadas_year_int.append(int(i))
    except:
        temporadas_year_int.append(i)

df_stats_players['Temporada'] = temporadas_year_int

#### Porteros

In [54]:
df_stats_gk['Temporada'] = [re.sub(r"\d\d/\d\d", '20'+str(i)[-2:], str(i)) for i in df_stats_gk['Temporada']]

temporadas_year_int = []
for i in df_stats_gk['Temporada']:
    try:
        temporadas_year_int.append(int(i))
    except:
        temporadas_year_int.append(i)

df_stats_gk['Temporada'] = temporadas_year_int

<div id='id5' />

## Nombre

Si existiese algún jugador del cual no se hubiese extraído bien el nombre, lo eliminamos del DataFrame dado que desconoceríamos su identidad

In [55]:
df_stats_players['Nombre'].fillna('No veo el nombre',inplace=True)
df_stats_gk['Nombre'].fillna('No veo el nombre',inplace=True)

df_stats_players = df_stats_players[df_stats_players['Nombre']!= 'No veo el nombre']
df_stats_gk = df_stats_gk[df_stats_gk['Nombre']!= 'No veo el nombre']

Usamos una expresión regular para sacar el nombre de cada jugador limpio

In [56]:
df_stats_gk['Nombre']=[re.compile(r'[^\n\s].*').findall(i)[-1] for i in df_stats_gk['Nombre']]
df_stats_players['Nombre'] = [re.compile(r'[^\n\s].*').findall(i)[-1] for i in df_stats_players['Nombre']]

<div id='id6' />

## Internacionalidades/Goles

Vamos a dividir en dos columnas:
* Internacionalidades
* Goles Selección

Los jugadores que no han tenido internacionalidades = "0 , 0"

In [57]:
df_stats_gk['Internacionalidades'].fillna("0 , 0", inplace=True)
df_stats_players['Internacionalidades'].fillna("0 , 0", inplace=True)

Internacionalidades de los porteros

In [58]:
partidos_int = [re.compile(r'\d\d*').findall(str(i))[0] for i in df_stats_gk['Internacionalidades']]
goles_int = [re.compile(r'\d\d*').findall(str(i))[-1] for i in df_stats_gk['Internacionalidades']]

In [59]:
df_stats_gk['Internacionalidades'] = partidos_int
df_stats_gk['Goles_Selección'] = goles_int

Internacionalidades del resto de jugadores

In [60]:
partidos_int = [re.compile(r'\d\d*').findall(str(i))[0] for i in df_stats_players['Internacionalidades']]
goles_int = [re.compile(r'\d\d*').findall(str(i))[-1] for i in df_stats_players['Internacionalidades']]

In [61]:
df_stats_players['Internacionalidades'] = partidos_int
df_stats_players['Goles_Selección'] = goles_int

In [62]:
df_stats_players

,Temporada,Nombre,Internacionalidades,Liga,Club,Partidos_Jugados,Goles,Asistencias,Yellow_Cards,Minutos_Jugados,Goles_Selección
0,2023,Romelu Lukaku,106,Serie A,Inter de Milán,19,5,3,1,1.134',72
1,2022,Romelu Lukaku,106,Premier League,Chelsea FC,26,8,1,1,1.583',72
2,2021,Romelu Lukaku,106,Serie A,Inter de Milán,36,24,10,4,2.887',72
3,2020,Romelu Lukaku,106,Serie A,Inter de Milán,36,23,2,2,2.983',72
4,2019,Romelu Lukaku,106,Premier League,Manchester United,32,12,1,4,2.130',72
...,...,...,...,...,...,...,...,...,...,...,...
102602,2020,Kirill Pechenin,18,Vysheyshaya Liga,Dynamo Brest,23,1,3,3,1.797',0
102603,2019,Kirill Pechenin,18,Vysheyshaya Liga,FK Vitebsk,29,2,4,2,2.356',0
102604,2018,Kirill Pechenin,18,Vysheyshaya Liga,FK Vitebsk,29,3,3,-,1.237',0
102605,2017,Kirill Pechenin,18,Vysheyshaya Liga,Naftan Novopolotsk,13,1,4,1,960',0


<div id='id7' />

## Minutos Jugados

Queremos el valor como entero

In [63]:
df_stats_players['Minutos_Jugados'].fillna(0, inplace=True)
df_stats_gk['Minutos_Jugados'].fillna(0, inplace=True)


Hacemos las siguientes modificaciones:
* Teníamos algunos - que querían funcionar como un 0
* Quitamos los . que marcan los miles
* Quitamos la ' de minutos, dado que los queremos como un valor entero

In [64]:
minutos_gk = [int(str(i).replace('-','0').replace('.','').replace("'","")) for i in df_stats_gk['Minutos_Jugados']]
minutos_player  = [int(str(i).replace('-','0').replace('.','').replace("'","")) for i in df_stats_players['Minutos_Jugados']]

df_stats_gk['Minutos_Jugados'] = minutos_gk
df_stats_players['Minutos_Jugados'] = minutos_player

<div id='id8' />

## Últimos retoques al DataFrame

En vez de tener 0s, en ciertas columnas tenemos guiones. Los reemplazamos

In [65]:
df_stats_players.replace('-',0,inplace=True)
df_stats_gk.replace('-',0,inplace=True)

En el DataFrame de los porteros, tanto la columna de Altura como la de competición no nos van a aportar nada (en la primera, debido a la cantidad de NaN que encontramos, y la segunda, debido a que solo ofrece un valor)

* Las quitamos

In [66]:
df_stats_gk.drop(['Altura','Competicion'], axis=1, inplace=True)

<div id='id9' />

## Vemos como quedan los DataFrames limpios

In [67]:
df_stats_players

,Temporada,Nombre,Internacionalidades,Liga,Club,Partidos_Jugados,Goles,Asistencias,Yellow_Cards,Minutos_Jugados,Goles_Selección
0,2023,Romelu Lukaku,106,Serie A,Inter de Milán,19,5,3,1,1134,72
1,2022,Romelu Lukaku,106,Premier League,Chelsea FC,26,8,1,1,1583,72
2,2021,Romelu Lukaku,106,Serie A,Inter de Milán,36,24,10,4,2887,72
3,2020,Romelu Lukaku,106,Serie A,Inter de Milán,36,23,2,2,2983,72
4,2019,Romelu Lukaku,106,Premier League,Manchester United,32,12,1,4,2130,72
...,...,...,...,...,...,...,...,...,...,...,...
102602,2020,Kirill Pechenin,18,Vysheyshaya Liga,Dynamo Brest,23,1,3,3,1797,0
102603,2019,Kirill Pechenin,18,Vysheyshaya Liga,FK Vitebsk,29,2,4,2,2356,0
102604,2018,Kirill Pechenin,18,Vysheyshaya Liga,FK Vitebsk,29,3,3,0,1237,0
102605,2017,Kirill Pechenin,18,Vysheyshaya Liga,Naftan Novopolotsk,13,1,4,1,960,0


In [68]:
df_stats_gk

,Temporada,Liga,Club,Partidos_Jugados,Goles_en_contra,Porterias_a_cero,Minutos_Jugados,Nombre,Internacionalidades,Goles_Selección
0,2023,Premier League,Chelsea FC,9,12,1,764,Edouard Mendy,29,0
1,2022,Premier League,Chelsea FC,34,31,14,3060,Edouard Mendy,29,0
2,2021,Premier League,Chelsea FC,31,25,16,2746,Edouard Mendy,29,0
3,2021,Ligue 1,Stade Rennais FC,1,1,0,90,Edouard Mendy,29,0
4,2020,Ligue 1,Stade Rennais FC,24,19,9,2160,Edouard Mendy,29,0
...,...,...,...,...,...,...,...,...,...,...
7880,2016,LaLiga SmartBank,SD Huesca,15,22,1,1304,Óscar Whalley,1,0
7881,2015,Promoción a LaLiga,Real Zaragoza,1,3,0,90,Óscar Whalley,1,0
7882,2015,LaLiga SmartBank,Real Zaragoza,19,28,4,1710,Óscar Whalley,1,0
7883,2014,LaLiga SmartBank,Real Zaragoza,2,2,0,180,Óscar Whalley,1,0


<div id='id10' />

## Unimos los DataFrames de Stats en uno

In [69]:
df_stats = df_stats_players.merge(df_stats_gk, how = 'outer').fillna(0)
df_stats

,Temporada,Nombre,Internacionalidades,Liga,Club,Partidos_Jugados,Goles,Asistencias,Yellow_Cards,Minutos_Jugados,Goles_Selección,Goles_en_contra,Porterias_a_cero
0,2023,Romelu Lukaku,106,Serie A,Inter de Milán,19,5,3,1,1134,72,0,0
1,2022,Romelu Lukaku,106,Premier League,Chelsea FC,26,8,1,1,1583,72,0,0
2,2021,Romelu Lukaku,106,Serie A,Inter de Milán,36,24,10,4,2887,72,0,0
3,2020,Romelu Lukaku,106,Serie A,Inter de Milán,36,23,2,2,2983,72,0,0
4,2019,Romelu Lukaku,106,Premier League,Manchester United,32,12,1,4,2130,72,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110226,2016,Óscar Whalley,1,LaLiga SmartBank,SD Huesca,15,0,0,0,1304,0,22,1
110227,2015,Óscar Whalley,1,Promoción a LaLiga,Real Zaragoza,1,0,0,0,90,0,3,0
110228,2015,Óscar Whalley,1,LaLiga SmartBank,Real Zaragoza,19,0,0,0,1710,0,28,4
110229,2014,Óscar Whalley,1,LaLiga SmartBank,Real Zaragoza,2,0,0,0,180,0,2,0


<div id='id11' />

### Hay que añadir a los DataFrames
* Posición del Jugador
* Nacionalidad

Para ello, recurrimos al dataset extraído previamente con BeautifulSoup

In [70]:
df_datos_subidas = pd.read_excel('../WebScraping/Beautiful Soup/data/ValoresDeMercado_Subidas.xlsx',index_col=0).reset_index(drop=True)
df_datos_bajadas = pd.read_excel('../WebScraping/Beautiful Soup/data/ValoresDeMercado_Bajadas.xlsx',index_col=0).reset_index(drop=True)
df_datos = pd.concat([df_datos_subidas,df_datos_bajadas],axis=0)

#### Problema: Hay 108 jugadores que tienen el nombre duplicado

In [71]:
lista_nombres_repetidos = list(df_datos[df_datos['Nombre'].duplicated()].Nombre.unique())


In [72]:
lista_nombres_repetidos = list(df_datos[df_datos['Nombre'].duplicated()].Nombre.unique())
print('Hay ', str(len(lista_nombres_repetidos)),' nombres similares entre distintos jugadores')

Hay  90  nombres similares entre distintos jugadores


### Los haremos a mano

Dado que son pocos, le añadimos un id diferente a cada uno de los jugadores con nombre similar

In [73]:
duplicados = []
for i in df_datos.Nombre:
    if i in lista_nombres_repetidos:
        duplicados.append('Repetido')
    else:
        duplicados.append('No Repetido')

df_datos['Duplicado'] = duplicados
#df_datos.to_excel('df_datos.xlsx') 

In [74]:
duplicados = []
for i in df_market.Nombre:
    if i in lista_nombres_repetidos:
        duplicados.append('Repetido')
    else:
        duplicados.append('No Repetido')

df_market['Duplicado'] = duplicados
#df_market.to_excel('df_market.xlsx') 

In [75]:
duplicados = []
for i in df_stats.Nombre:
    if i in lista_nombres_repetidos:
        duplicados.append('Repetido')
    else:
        duplicados.append('No Repetido')

df_stats['Duplicado'] = duplicados
#df_stats.to_excel('df_stats.xlsx') 

Una vez hemos hecho los cambios desde excel, los importamos

In [12]:
df_datos = pd.read_excel('data_repaired/df_datos.xlsx',index_col=0).fillna('0')
df_market = pd.read_excel('data_repaired/df_market.xlsx',index_col=0).fillna('0')
df_stats = pd.read_excel('data_repaired/df_stats.xlsx',index_col=0).fillna('0')

Y ahora sí, añadimos las columnas de nacionalidad y posición a Stats y Market

In [13]:
df_stats = df_stats.merge(df_datos[['Nombre','Nombre_id','Posición','Nacionalidad 1','Nacionalidad 2']], how='inner', on=['Nombre','Nombre_id'])

In [14]:
df_market = df_market.merge(df_datos[['Nombre','Nombre_id','Posición','Nacionalidad 1','Nacionalidad 2']], how='inner', on=['Nombre','Nombre_id'])

### Quitamos las posiciones "0"

In [15]:
df_market = df_market[df_market['Posición']!='0'].reset_index(drop=True)
df_stats = df_stats[df_stats['Posición']!='0'].reset_index(drop=True)

### El ID de cada jugador cuyo nombre no se repita, será su propio nombre


En el DataFrame de mercado

In [16]:
ids_players = []
for i in range(len(df_market['Nombre_id'])):
    try:
        if df_market['Nombre_id'][i] == "0":
            ids_players.append(df_market['Nombre'][i])
        elif df_market['Nombre_id'][i] != "0":
            ids_players.append(df_market['Nombre_id'][i])
    except:
        continue

df_market['Nombre_id'] = ids_players


En el DataFrame de Estadísticas

In [17]:
ids_players = []
for i in range(len(df_stats['Nombre_id'])):
    try:
        if df_stats['Nombre_id'][i] == "0":
            ids_players.append(df_stats['Nombre'][i])
        elif df_stats['Nombre_id'][i] != "0":
            ids_players.append(df_stats['Nombre_id'][i])
    except:
        continue

df_stats['Nombre_id'] = ids_players

<div id='id12' />

## Exportamos lo que nos interesa 

Estadísticas

In [18]:
df_stats.columns

Index(['Temporada', 'Nombre', 'Nombre_id', 'Internacionalidades', 'Liga',
       'Club', 'Partidos_Jugados', 'Goles', 'Asistencias', 'Yellow_Cards',
       'Minutos_Jugados', 'Goles_Selección', 'Goles_en_contra',
       'Porterias_a_cero', 'Duplicado', 'Posición', 'Nacionalidad 1',
       'Nacionalidad 2'],
      dtype='object')

In [19]:
df_stats[['Temporada', 'Nombre','Nombre_id', 'Internacionalidades', 'Liga', 'Club',
       'Partidos_Jugados', 'Goles', 'Asistencias', 'Yellow_Cards',
       'Minutos_Jugados', 'Goles_Selección', 'Goles_en_contra',
       'Porterias_a_cero','Posición', 'Nacionalidad 1',
       'Nacionalidad 2']].to_csv('../Data_Processed/Stats.csv')

Mercado

In [20]:
df_market.columns

Index(['Nombre', 'Nombre_id', 'Fecha de actualización', 'Valor de mercado',
       'Club', 'Edad', 'Diferencia_Valor', 'Club_Valor',
       'Año de actualización', 'Mes', 'Resta', 'Temporada', 'Duplicado',
       'Posición', 'Nacionalidad 1', 'Nacionalidad 2'],
      dtype='object')

In [21]:
df_market[['Nombre','Posición','Club', 'Edad', 'Nacionalidad 1', 'Nacionalidad 2',  'Valor de mercado',
        'Diferencia_Valor', 'Club_Valor', 'Año de actualización', 'Temporada',
       'Nombre_id', 'Fecha de actualización'
       ]].to_csv('../Data_Processed/Market_Value.csv')